# Project Three Notebook
By [Erica Dumore](https://www.linkedin.com/in/erica-dumore-62b306a2/) 


## Is this restaurant up to code?

This project takes generated data from [data.gov](data.gov) that evaluated different restaurants located throughout San Francisco, California and reported their health inspection spore. The city released this data publically, reporting the name of a restaurant, the date of the inspection, the reason for the inspection, the inspection score it received, a description if a violation was found, and the risk category the restaurant was placed in as a result. 

This project focuses on specific categories, inspection score and inspection type, found within [this csv file](restaurant.csv) which can be downloaded from this [link](https://catalog.data.gov/dataset/food-inspections-lives-standard), to predict whether our target, the inspection scores, can be predicted by our features, the different types of inspections being preformed. Through the use of a linear regression graph, we will see what correlation exists between our target and our features. Based on the pattern generated, we will be able to predict whether an inspection score does correlate with the type of inspection being preformed, allowing the investigator to have an idea prior to preforming the inspection, what they can expect to see. 

The idea being that a new ownership inspection may often result in a high inspection score with low risk, verse a foodborne illness investigation may result in a low inspection score with high risk. Or a scheduled, verse an unscheduled inspection, may influence the outcome of the inspection score and risk category. These were a few of my hypotheses at the outset of this project. 

## Process

This project took a lot of trial and error. The first step was deciding what my target and features were going to be. I knew I wanted the inspection scores to be my target, but deciding what would be my features was a more difficult task. Ultimately I decided to use the different types of inspections being performed. This was because I needed a feature that was going to be predictory, something that could be used repetitively based off of one factor. 

After I had decided my target and my feature I created my first set of code. The first time I created my code I mistakenly generated a cell for each individual linear graph for each different type of inspection performed. Although these graphs were accurate and did provide me with some feedback, they were too specific and did not provide me with an overall view of how all the different types of inspections correlated together in terms of what range they fell into for inspection scores. 

Below, under the heading Data Cleaning, please find an explanation of how I rewrote my code to solve the above mentioned problem, utilizing dummy variables to add all of the different types of inspections onto my data spreadsheet to generate number responses, making it even easier to graph. 

In [274]:
import os
try:
    inputFunc = raw_input
except NameError:
    inputFunc = input

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import numpy as np
 
import seaborn as sns
from statsmodels.formula.api import ols

from sklearn import linear_model
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from patsy import dmatrices

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import random



# Custom functions

def evaluate(pred, labels_test):
    acc = accuracy_score(pred, labels_test)
    print ("Accuracey: %s"%acc)
    tn, fp, fn, tp = confusion_matrix(labels_test, pred).ravel()

    recall = tp / (tp + fp)
    percision = tp / (tp + fn)
    f1 = (2 / ((1/recall)+(1/percision)))

    print ("")
    print ("True Negatives: %s"%tn)
    print ("False Positives: %s"%fp)
    print ("False Negatives: %s"%fn)
    print ("True Positives: %s"%tp)
    print ("Recall: %s"%recall)
    print ("Precision: %s"%percision)
    print ("F1 Score: %s"%f1)

def plot_bound(Z_val,data,col1,col2,binary):
    # Z-val equals "Yes" value. E.g., "Y" or "1". 
    # data equals df
    # col1 and col2 defines which colums to use from data
    # Plot binary decision boundary. 
    # For this, we will assign a color to each
    # point in the mesh [x_min, m_max]x[y_min, y_max].
    
    x_min = float(data.iloc[:,[col1]].min())-float(data.iloc[:,[col1]].min())*0.10 
    x_max = float(data.iloc[:,[col1]].max()+float(data.iloc[:,[col1]].min())*0.10)
    y_min = 0.0; 
    y_max = float(training.iloc[:,[col2]].max())+float(training.iloc[:,[col2]].max())*0.10
    h_x = (x_max-x_min)/100  # step size in the mesh
    h_y = (y_max-y_min)/100  # step size in the mesh
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h_x), np.arange(y_min, y_max, h_y))
    if binary == 1:
        Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])   
        Z = np.where(Z=="Y",1,0)
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]
    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.pcolormesh(xx, yy, Z)
    plt.show()

## Data Cleaning

Below I have loaded the data collected and begun to feed it to the statistical model. Utilizing my target, inspection scores, and my features, types of inspections. Here I have loaded my [csv file](restaurant.csv) from: https://catalog.data.gov/dataset/food-inspections-lives-standard 


In [275]:
# This is my raw data before cleaning it at all. 
raw_data_df = pd.read_csv('restaurant.csv', parse_dates=[0]) 
raw_data_df.head()

business_id                    business_name  business_address  \
0       90604              Taqueria Los Amigos  5300 Mission St.   
1        1757  Dar Bar Pakistani/Indian Cusine      1412 Polk St   
2        4864        DRAGON CITY BAKERY & CAFE   2367 MISSION St   
3       79782                          Deli 23      2449 23rd St   
4       76437                  Sweetheart Cafe     909 Grant Ave   

   business_city business_state business_postal_code  business_latitude  \
0  San Francisco             CA                94112                NaN   
1  San Francisco             CA                94109          37.789784   
2  San Francisco             CA                94110          37.759174   
3  San Francisco             CA                94110                NaN   
4  San Francisco             CA                94108                NaN   

   business_longitude         business_location  business_phone_number  \
0                 NaN                       NaN                    NaN   
1         -122.420455  (37.789784, -122.420455)                    NaN   
2         -122.419066  (37.759174, -122.419066)           1.415583e+10   
3                 NaN                       NaN                    NaN   
4                 NaN                       NaN                    NaN   

    inspection_id inspection_date  inspection_score        inspection_type  \
0  90604_20170502     5/2/17 0:00               NaN          New Ownership   
1   1757_20170928    9/28/17 0:00              86.0  Routine - Unscheduled   
2   4864_20161206    12/6/16 0:00              84.0  Routine - Unscheduled   
3  79782_20160503     5/3/16 0:00              92.0  Routine - Unscheduled   
4  76437_20160329    3/29/16 0:00              76.0  Routine - Unscheduled   

            violation_id                              violation_description  \
0  90604_20170502_103133             Foods not protected from contamination   
1   1757_20170928_103131                   Moderate risk vermin infestation   
2   4864_20161206_103157  Food safety certificate or food handler card n...   
3  79782_20160503_103120             Moderate risk food holding temperature   
4  76437_20160329_103113                 Sewage or wastewater contamination   

   risk_category  
0  Moderate Risk  
1  Moderate Risk  
2       Low Risk  
3  Moderate Risk  
4      High Risk

In [276]:
# Utilizing the .unique() feature I filtered out the different types of inspection types.
print(raw_data_df["inspection_type"].unique())

['New Ownership' 'Routine - Unscheduled' 'Structural Inspection'
 'Reinspection/Followup' 'New Construction' 'Complaint'
 'Non-inspection site visit' 'Routine - Scheduled'
 'Foodborne Illness Investigation' 'Complaint Reinspection/Followup'
 'Special Event' 'New Ownership - Followup' 'Community Health Assessment'
 'Administrative or Document Review']


In [277]:
# Utilizing the drop feature, I dropped the columns that did not apply to this project. 
raw_data_df = raw_data_df.drop([ 'business_id',
                                            'business_address',
                                            'business_city',
                                            'business_state',
                                            'business_postal_code',
                                            'business_latitude',
                                            'business_longitude',
                                            'business_location',
                                            'business_phone_number',
                                            'inspection_id',
                                            'inspection_date',
                                            'violation_id',
                                            'violation_description',
                                            'risk_category'
                                                                       ], 1)
raw_data_df.head()

business_name  inspection_score        inspection_type
0              Taqueria Los Amigos               NaN          New Ownership
1  Dar Bar Pakistani/Indian Cusine              86.0  Routine - Unscheduled
2        DRAGON CITY BAKERY & CAFE              84.0  Routine - Unscheduled
3                          Deli 23              92.0  Routine - Unscheduled
4                  Sweetheart Cafe              76.0  Routine - Unscheduled

In [278]:
# Utilizing the renaming feature, I renamed the business column.
raw_data_df = raw_data_df.rename(columns={
                                                        'business_name': 'Business', 
                                                        'inspection_score': 'Score',
                                                        'inspection_type': 'Type',
                                                     })
raw_data_df.head()

Business  Score                   Type
0              Taqueria Los Amigos    NaN          New Ownership
1  Dar Bar Pakistani/Indian Cusine   86.0  Routine - Unscheduled
2        DRAGON CITY BAKERY & CAFE   84.0  Routine - Unscheduled
3                          Deli 23   92.0  Routine - Unscheduled
4                  Sweetheart Cafe   76.0  Routine - Unscheduled

In [279]:
# Utilizing the dummies feature I added the different types of inspections to the end of the data set as features.
df_new = pd.concat([raw_data_df, pd.get_dummies(raw_data_df['Type'])], axis=1)

df_new.head()

Business  Score                   Type  \
0              Taqueria Los Amigos    NaN          New Ownership   
1  Dar Bar Pakistani/Indian Cusine   86.0  Routine - Unscheduled   
2        DRAGON CITY BAKERY & CAFE   84.0  Routine - Unscheduled   
3                          Deli 23   92.0  Routine - Unscheduled   
4                  Sweetheart Cafe   76.0  Routine - Unscheduled   

   Administrative or Document Review  Community Health Assessment  Complaint  \
0                                  0                            0          0   
1                                  0                            0          0   
2                                  0                            0          0   
3                                  0                            0          0   
4                                  0                            0          0   

   Complaint Reinspection/Followup  Foodborne Illness Investigation  \
0                                0                                0   
1                                0                                0   
2                                0                                0   
3                                0                                0   
4                                0                                0   

   New Construction  New Ownership  New Ownership - Followup  \
0                 0              1                         0   
1                 0              0                         0   
2                 0              0                         0   
3                 0              0                         0   
4                 0              0                         0   

   Non-inspection site visit  Reinspection/Followup  Routine - Scheduled  \
0                          0                      0                    0   
1                          0                      0                    0   
2                          0                      0                    0   
3                          0                      0                    0   
4                          0                      0                    0   

   Routine - Unscheduled  Special Event  Structural Inspection  
0                      0              0                      0  
1                      1              0                      0  
2                      1              0                      0  
3                      1              0                      0  
4                      1              0                      0

In [280]:
# Utilizing the dropna feature to drop any non-numeral responses that appear in the data set as "NaN."
raw_data_df = raw_data_df.dropna()
raw_data_df.head()

Business  Score                   Type
1  Dar Bar Pakistani/Indian Cusine   86.0  Routine - Unscheduled
2        DRAGON CITY BAKERY & CAFE   84.0  Routine - Unscheduled
3                          Deli 23   92.0  Routine - Unscheduled
4                  Sweetheart Cafe   76.0  Routine - Unscheduled
6                      Jelly Donut   88.0  Routine - Unscheduled

In [281]:
# Training the model to make a linear regression where the target, Score, is numeric 
score_lin_df = raw_data_df[[
                               'Score', 
                               'Type', 
                               ]].copy()
score_lin_df.head()

Score                   Type
1   86.0  Routine - Unscheduled
2   84.0  Routine - Unscheduled
3   92.0  Routine - Unscheduled
4   76.0  Routine - Unscheduled
6   88.0  Routine - Unscheduled

In [282]:
data = score_lin_df

data = data[data["Score"]<=100]

holdout = data.sample(frac=0.2)
training = data.loc[~data.index.isin(holdout.index)]

In [283]:
sns.lmplot(x="Type", y="Score", data=training, x_estimator=np.mean, order=1)

ValueError: could not convert string to float: 'Routine - Unscheduled'

ValueError: could not convert string to float: 'Routine - Unscheduled'

In [284]:
processed_data_df = raw_data_df
processed_data_df["Score"] = 0
processed_data_df.loc[processed_data_df['Type'] == 'Type'] = 1
processed_data_df = processed_data_df.dropna()
score_lin_df = processed_data_df[[
                               'Type'
                               ]].copy()
score_lin_df.head()
score_lin_df

Type
1      Routine - Unscheduled
2      Routine - Unscheduled
3      Routine - Unscheduled
4      Routine - Unscheduled
6      Routine - Unscheduled
7      Routine - Unscheduled
8      Routine - Unscheduled
10     Routine - Unscheduled
11     Routine - Unscheduled
12     Routine - Unscheduled
13     Routine - Unscheduled
14     Routine - Unscheduled
15     Routine - Unscheduled
16     Routine - Unscheduled
17     Routine - Unscheduled
18     Routine - Unscheduled
19     Routine - Unscheduled
20     Routine - Unscheduled
22     Routine - Unscheduled
24     Routine - Unscheduled
25     Routine - Unscheduled
26     Routine - Unscheduled
27     Routine - Unscheduled
28     Routine - Unscheduled
29     Routine - Unscheduled
30     Routine - Unscheduled
31     Routine - Unscheduled
32     Routine - Unscheduled
33     Routine - Unscheduled
34     Routine - Unscheduled
...                      ...
50529  Routine - Unscheduled
50532  Routine - Unscheduled
50533  Routine - Unscheduled
50534  Routine - Unscheduled
50535  Routine - Unscheduled
50537  Routine - Unscheduled
50538  Routine - Unscheduled
50539  Routine - Unscheduled
50540  Routine - Unscheduled
50541  Routine - Unscheduled
50542  Routine - Unscheduled
50546  Routine - Unscheduled
50547  Routine - Unscheduled
50549  Routine - Unscheduled
50550  Routine - Unscheduled
50551  Routine - Unscheduled
50553  Routine - Unscheduled
50554  Routine - Unscheduled
50555  Routine - Unscheduled
50556  Routine - Unscheduled
50557  Routine - Unscheduled
50558  Routine - Unscheduled
50559  Routine - Unscheduled
50560  Routine - Unscheduled
50561  Routine - Unscheduled
50562  Routine - Unscheduled
50563  Routine - Unscheduled
50564  Routine - Unscheduled
50565  Routine - Unscheduled
50566  Routine - Unscheduled

[37714 rows x 1 columns]

In [285]:
# Further generating results based off of the model above to provide more information based off of the linear regression
model = ols("Score ~  Type", training).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Score   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.390
Date:                Mon, 04 Dec 2017   Prob (F-statistic):              0.122
Time:                        11:16:42   Log-Likelihood:            -1.0996e+05
No. Observations:               30171   AIC:                         2.199e+05
Df Residuals:                   30169   BIC:                         2.199e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                       100.0000      9.261     10.798      0.000      81.849     118.151
Type[T.Routine - Unscheduled]   -14.3165      9.261     -1.546      0.122     -32.468       3.835
==============================================================================
Omnibus:                     2133.196   Durbin-Watson:                   1.952
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2610.059
Skew:                          -0.709   Prob(JB):                         0.00
Kurtosis:                       3.252   Cond. No.                         347.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [286]:
# Rerun with SciKitLearn because it's easy to check accuracy
features_train = training.drop("Score", axis=1).as_matrix(columns=None)
labels_train = training["Score"].as_matrix(columns=None)

features_test = holdout.drop("Score", axis=1).as_matrix(columns=None)
labels_test = holdout["Score"].as_matrix(columns=None)

lm = linear_model.LinearRegression()
clf = lm.fit(features_train, labels_train)
pred = clf.predict(features_test)
accuracy = metrics.r2_score(labels_test, pred)
print("R squared:",lm.score(features_train,labels_train))
print("Accuracy:",accuracy)

ValueError: could not convert string to float: 'Routine - Unscheduled'